In [ ]:
from robovast.common.analysis import read_output_files, read_output_csv, get_behavior_info
import pandas as pd
from robovast_nav.gui import MapVisualizer
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import numpy as np
import os
DATA_DIR = ''

df = read_output_files(DATA_DIR, lambda test_dir: read_output_csv(test_dir, "poses.csv"))

# Generate colors for each test
tests = df['test'].unique()
variants = df['variant'].unique()
colors = cm.rainbow(np.linspace(0, 1, len(tests)))

df_behaviors = read_output_files(DATA_DIR, lambda test_dir: read_output_csv(test_dir, "behaviors.csv"))
df_behavior_info = get_behavior_info('differential_drive_robot.nav_through_poses', df_behaviors)

## Overview

In [ ]:
# Multi-Run Comparison: All Robot Paths by Variant

# Generate colors for variants (not tests)
variant_colors = cm.rainbow(np.linspace(0, 1, len(variants)))

# Create separate visualization for each variant
num_variants = len(variants)
cols = min(3, num_variants)
rows = (num_variants + cols - 1) // cols

fig, axes = plt.subplots(rows, cols, figsize=(7*cols, 7*rows))
if num_variants == 1:
    axes = np.array([axes])
axes = axes.flatten()

for i, variant in enumerate(variants):
    df_variant = df[df['variant'] == variant]
    df_gt_mask = df_variant['frame'] == 'turtlebot4_base_link_gt'
    map_file0 = df_variant['map_file'].iloc[0]
    variant0 = df_variant['variant'].iloc[0]
    test0 = df_variant['test'].iloc[0]
    map_path = os.path.join(DATA_DIR, variant0, test0, map_file0)
    
    # Create visualizer for this variant
    viz = MapVisualizer()
    viz.load_map(map_path)
    viz.create_figure(ax=axes[i], figsize=(7, 7))
    
    # Draw all tests for this variant
    for test in tests:
        df_test_variant = df_variant[df_variant['test'] == test]
        path_robot = list(zip(df_test_variant.loc[df_gt_mask, 'position.x'], 
                             df_test_variant.loc[df_gt_mask, 'position.y']))
        if len(path_robot) > 0:
            viz.draw_path(path_robot, color=variant_colors[i], linewidth=1.5, 
                         alpha=0.6, show_endpoints=False)
    
    viz.ax.set_title(f'{variant}\n({len(tests)} Tests)', 
                     fontsize=12, fontweight='bold')

# Hide empty subplots
for j in range(i+1, len(axes)):
    axes[j].axis('off')

plt.tight_layout()
plt.show()


## Navigation Duration Analysis

In [ ]:
# Multi-Variant + Multi-Test Navigation Duration Analysis
fig, axes = plt.subplots(2, 3, figsize=(20, 12))

# Use df_behavior_info directly - it already has one row per test-variant combination
df_duration_summary = df_behavior_info[['variant', 'test', 'duration']].copy()

# Create a pivot table for easier access
duration_pivot = df_duration_summary.pivot(index='variant', columns='test', values='duration')

# Sort variants to match our color scheme
duration_pivot = duration_pivot.reindex(variants)

# 1. Grouped Bar Chart - Duration by Variant and Test
ax = axes[0, 0]
x = np.arange(len(tests))
width = 0.12  # Width of bars
offsets = np.linspace(-3*width, 3*width, len(variants))

sorted_tests_list = sorted([str(t) for t in tests])
for i, variant in enumerate(variants):
    variant_data = df_duration_summary[df_duration_summary['variant'] == variant].copy()
    if len(variant_data) > 0:
        # Create array with NaN for missing tests
        durations = []
        for test in sorted_tests_list:
            test_data = variant_data[variant_data['test'] == test]
            if len(test_data) > 0:
                durations.append(test_data['duration'].values[0])
            else:
                durations.append(np.nan)
        
        # Only plot non-NaN values
        durations = np.array(durations)
        valid_mask = ~np.isnan(durations)
        if valid_mask.any():
            ax.bar(x[valid_mask] + offsets[i], durations[valid_mask], width, label=variant, 
                   color=variant_colors[i], alpha=0.8, edgecolor='black', linewidth=0.8)

ax.set_xlabel('Test Number', fontsize=11, fontweight='bold')
ax.set_ylabel('Duration (seconds)', fontsize=11, fontweight='bold')
ax.set_title('Navigation Duration: Variants × Tests', fontsize=12, fontweight='bold')
ax.set_xticks(x)
ax.set_xticklabels(sorted_tests_list)
ax.legend(fontsize=8, ncol=2, loc='upper left')
ax.grid(True, alpha=0.3, axis='y', linewidth=0.5)

# 2. Box Plot - Duration Distribution by Variant
ax = axes[0, 1]
variant_duration_list = [df_duration_summary[df_duration_summary['variant'] == v]['duration'].values 
                         for v in variants]
bp = ax.boxplot(variant_duration_list, tick_labels=variants, patch_artist=True, widths=0.6)

for patch, color in zip(bp['boxes'], variant_colors):
    patch.set_facecolor(color)
    patch.set_alpha(0.7)
    patch.set_edgecolor('black')
    patch.set_linewidth(1.5)

for element in ['whiskers', 'fliers', 'medians', 'caps']:
    plt.setp(bp[element], color='black', linewidth=1.5)

ax.set_xlabel('Variant', fontsize=11, fontweight='bold')
ax.set_ylabel('Duration (seconds)', fontsize=11, fontweight='bold')
ax.set_title('Duration Distribution by Variant', fontsize=12, fontweight='bold')
ax.grid(True, alpha=0.3, axis='y', linewidth=0.5)
plt.setp(ax.xaxis.get_majorticklabels(), rotation=45, ha='right')

# 3. Heatmap - Duration across Variants and Tests
ax = axes[0, 2]
duration_matrix = duration_pivot.values
im = ax.imshow(duration_matrix, cmap='RdYlGn_r', aspect='auto')
ax.set_xticks(np.arange(len(sorted_tests_list)))
ax.set_yticks(np.arange(len(variants)))
ax.set_xticklabels(sorted_tests_list)
ax.set_yticklabels(variants)
ax.set_xlabel('Test Number', fontsize=11, fontweight='bold')
ax.set_ylabel('Variant', fontsize=11, fontweight='bold')
ax.set_title('Duration Heatmap: Variants × Tests', fontsize=12, fontweight='bold')

# Add colorbar
cbar = plt.colorbar(im, ax=ax)
cbar.set_label('Duration (s)', fontsize=10, fontweight='bold')

# Add text annotations
if duration_matrix.size > 0 and len(sorted_tests_list) > 0:
    for i in range(len(variants)):
        for j in range(len(sorted_tests_list)):
            if not np.isnan(duration_matrix[i, j]):
                text = ax.text(j, i, f'{duration_matrix[i, j]:.1f}',
                              ha="center", va="center", color="black", fontsize=7, fontweight='bold')

# 4. Mean Duration by Variant with Error Bars
ax = axes[1, 0]
variant_means = [df_duration_summary[df_duration_summary['variant'] == v]['duration'].mean() 
                 for v in variants]
variant_stds = [df_duration_summary[df_duration_summary['variant'] == v]['duration'].std() 
                for v in variants]
x_pos = np.arange(len(variants))

bars = ax.bar(x_pos, variant_means, yerr=variant_stds, capsize=5, 
              alpha=0.75, color=variant_colors, edgecolor='black', linewidth=1.5)
ax.set_xlabel('Variant', fontsize=11, fontweight='bold')
ax.set_ylabel('Mean Duration (seconds)', fontsize=11, fontweight='bold')
ax.set_title('Mean Duration by Variant (±1 SD)', fontsize=12, fontweight='bold')
ax.set_xticks(x_pos)
ax.set_xticklabels(variants)
ax.grid(True, alpha=0.3, axis='y', linewidth=0.5)
plt.setp(ax.xaxis.get_majorticklabels(), rotation=45, ha='right')

# Add value labels
for i, (mean, std) in enumerate(zip(variant_means, variant_stds)):
    ax.text(i, mean + std + 0.5, f'{mean:.1f}s', ha='center', va='bottom', 
            fontsize=9, fontweight='bold')

# 5. Coefficient of Variation by Variant
ax = axes[1, 1]
variant_cv = [(df_duration_summary[df_duration_summary['variant'] == v]['duration'].std() / 
               df_duration_summary[df_duration_summary['variant'] == v]['duration'].mean()) * 100 
              for v in variants]

bars = ax.bar(x_pos, variant_cv, alpha=0.75, color=variant_colors, 
              edgecolor='black', linewidth=1.5)
ax.set_xlabel('Variant', fontsize=11, fontweight='bold')
ax.set_ylabel('Coefficient of Variation (%)', fontsize=11, fontweight='bold')
ax.set_title('Duration Variability by Variant', fontsize=12, fontweight='bold')
ax.set_xticks(x_pos)
ax.set_xticklabels(variants)
ax.grid(True, alpha=0.3, axis='y', linewidth=0.5)
plt.setp(ax.xaxis.get_majorticklabels(), rotation=45, ha='right')

# Add value labels
for i, cv in enumerate(variant_cv):
    ax.text(i, cv + 0.2, f'{cv:.1f}%', ha='center', va='bottom', 
            fontsize=9, fontweight='bold')

# 6. Violin Plot - Duration Distribution by Variant
ax = axes[1, 2]

# Filter out empty arrays and track valid positions
valid_data = []
valid_positions = []
valid_colors = []
for i, (data, variant) in enumerate(zip(variant_duration_list, variants)):
    if len(data) > 0:
        valid_data.append(data)
        valid_positions.append(i)
        valid_colors.append(variant_colors[i])

if len(valid_data) > 0:
    parts = ax.violinplot(valid_data, positions=valid_positions, 
                          showmeans=True, showmedians=True)

    for i, pc in enumerate(parts['bodies']):
        pc.set_facecolor(valid_colors[i])
        pc.set_alpha(0.7)
        pc.set_edgecolor('black')
        pc.set_linewidth(1.5)

    plt.setp(parts['cmeans'], color='red', linewidth=2)
    plt.setp(parts['cmedians'], color='blue', linewidth=2)
    plt.setp(parts['cbars'], color='black', linewidth=1.5)
    plt.setp(parts['cmaxes'], color='black', linewidth=1.5)
    plt.setp(parts['cmins'], color='black', linewidth=1.5)

ax.set_xlabel('Variant', fontsize=11, fontweight='bold')
ax.set_ylabel('Duration (seconds)', fontsize=11, fontweight='bold')
ax.set_title('Duration Distribution Shape by Variant', fontsize=12, fontweight='bold')
ax.set_xticks(range(len(variants)))
ax.set_xticklabels(variants)
ax.grid(True, alpha=0.3, axis='y', linewidth=0.5)
plt.setp(ax.xaxis.get_majorticklabels(), rotation=45, ha='right')

plt.tight_layout()
plt.show()

# Print summary statistics
print("\n" + "="*80)
print("MULTI-VARIANT DURATION ANALYSIS SUMMARY")
print("="*80)
for variant in variants:
    durations = df_duration_summary[df_duration_summary['variant'] == variant]['duration'].values
    if len(durations) > 0:
        print(f"\n{variant}:")
        print(f"  Mean: {durations.mean():.2f}s (±{durations.std():.2f}s)")
        print(f"  Range: [{durations.min():.2f}s, {durations.max():.2f}s]")
        print(f"  CV: {(durations.std()/durations.mean())*100:.1f}%")
    else:
        print(f"\n{variant}:")
        print(f"  No data available")
print("="*80)

In [ ]:
# Prepare data for multi-variant + multi-run comparison
import numpy as np
import matplotlib.pyplot as plt

# Split by frame and prepare for all runs
df_robot = df[df['frame'] == 'base_link'].copy()
df_groundtruth = df[df['frame'] == 'turtlebot4_base_link_gt'].copy()

# Sort by variant, test and timestamp
df_robot.sort_values(['variant', 'test', 'timestamp'], inplace=True)
df_groundtruth.sort_values(['variant', 'test', 'timestamp'], inplace=True)

# print(f"Processing {len(variants)} variants × {len(tests)} tests = {len(variants)*len(tests)} runs")
# print(f"Robot data points: {len(df_robot)}")
# print(f"Ground truth data points: {len(df_groundtruth)}")

In [ ]:
# Calculate errors for each variant-test combination
from scipy.interpolate import interp1d

# Store results for each variant-test run
run_errors = {}
run_stats = []

for variant in variants:
    for test in tests:
        # Get data for this specific variant-test combination
        mask_robot = (df_robot['variant'] == variant) & (df_robot['test'] == test)
        mask_gt = (df_groundtruth['variant'] == variant) & (df_groundtruth['test'] == test)
        
        df_robot_run = df_robot[mask_robot].copy()
        df_gt_run = df_groundtruth[mask_gt].copy()
        
        if len(df_robot_run) == 0 or len(df_gt_run) == 0:
            continue
        
        # Find common time range
        time_start = max(df_robot_run['timestamp'].iloc[0], df_gt_run['timestamp'].iloc[0])
        time_end = min(df_robot_run['timestamp'].iloc[-1], df_gt_run['timestamp'].iloc[-1])
        
        # Create interpolation functions for ground truth
        gt_timestamps = df_gt_run['timestamp'].values
        gt_interp_x = interp1d(gt_timestamps, df_gt_run['position.x'].values, kind='linear', fill_value='extrapolate')
        gt_interp_y = interp1d(gt_timestamps, df_gt_run['position.y'].values, kind='linear', fill_value='extrapolate')
        gt_interp_yaw = interp1d(gt_timestamps, df_gt_run['orientation.yaw'].values, kind='linear', fill_value='extrapolate')
        
        # Filter robot data to common time range
        time_mask = (df_robot_run['timestamp'] >= time_start) & (df_robot_run['timestamp'] <= time_end)
        df_robot_aligned = df_robot_run[time_mask].copy()
        
        if len(df_robot_aligned) == 0:
            continue
        
        robot_timestamps = df_robot_aligned['timestamp'].values
        
        # Interpolate ground truth at robot timestamps
        gt_x = gt_interp_x(robot_timestamps)
        gt_y = gt_interp_y(robot_timestamps)
        gt_yaw = gt_interp_yaw(robot_timestamps)
        
        # Calculate position errors
        position_error_x = df_robot_aligned['position.x'].values - gt_x
        position_error_y = df_robot_aligned['position.y'].values - gt_y
        absolute_position_error = np.sqrt(position_error_x**2 + position_error_y**2)
        
        # Calculate orientation error
        orientation_error = df_robot_aligned['orientation.yaw'].values - gt_yaw
        orientation_error = np.arctan2(np.sin(orientation_error), np.cos(orientation_error))
        
        # Store errors for this run
        run_key = f"{variant}_{test}"
        run_errors[run_key] = {
            'variant': variant,
            'test': test,
            'timestamps': robot_timestamps,
            'position_error': absolute_position_error,
            'position_error_x': position_error_x,
            'position_error_y': position_error_y,
            'orientation_error': orientation_error
        }
        
        # Calculate statistics
        run_stats.append({
            'variant': variant,
            'test': test,
            'mean_pos_error': np.mean(absolute_position_error),
            'std_pos_error': np.std(absolute_position_error),
            'max_pos_error': np.max(absolute_position_error),
            'median_pos_error': np.median(absolute_position_error),
            'p95_pos_error': np.percentile(absolute_position_error, 95),
            'mean_orient_error': np.mean(np.abs(orientation_error)),
            'std_orient_error': np.std(orientation_error),
            'max_orient_error': np.max(np.abs(orientation_error))
        })

# Convert to DataFrame for easy analysis
stats_df = pd.DataFrame(run_stats)
# print(f"Calculated errors for {len(run_stats)} runs")

In [ ]:
from robovast.common.analysis import calculate_speeds_from_poses
df_gt_speeds = calculate_speeds_from_poses(df_groundtruth)

In [ ]:
# Speed Statistics by Variant and Test
speed_stats = []
for variant in variants:
    for test in tests:
        mask = (df_gt_speeds['variant'] == variant) & (df_gt_speeds['test'] == test)
        df_test_speeds = df_gt_speeds[mask]
        
        if len(df_test_speeds) == 0:
            continue
        
        linear_speeds = df_test_speeds['linear_speed'].values
        angular_speeds = df_test_speeds['angular_speed'].values
        
        speed_stats.append({
            'variant': variant,
            'test': test,
            'mean_linear': np.mean(linear_speeds),
            'max_linear': np.max(linear_speeds),
            'std_linear': np.std(linear_speeds),
            'median_linear': np.median(linear_speeds),
            'mean_angular': np.mean(np.abs(angular_speeds)),
            'max_angular': np.max(np.abs(angular_speeds)),
            'std_angular': np.std(angular_speeds),
            'median_angular': np.median(np.abs(angular_speeds))
        })

speed_stats_df = pd.DataFrame(speed_stats)
# print(f"Calculated speed stats for {len(speed_stats)} runs")

## Localization Errors

In [ ]:
# Multi-Variant Position Error Analysis
fig, axes = plt.subplots(2, 3, figsize=(20, 12))

# 1. Box Plot - Position Error by Variant
ax = axes[0, 0]
variant_pos_errors = [stats_df[stats_df['variant'] == v]['mean_pos_error'].values for v in variants]
bp = ax.boxplot(variant_pos_errors, tick_labels=variants, patch_artist=True, widths=0.6)

for patch, color in zip(bp['boxes'], variant_colors):
    patch.set_facecolor(color)
    patch.set_alpha(0.7)
    patch.set_edgecolor('black')
    patch.set_linewidth(1.5)

for element in ['whiskers', 'fliers', 'medians', 'caps']:
    plt.setp(bp[element], color='black', linewidth=1.5)

ax.set_xlabel('Variant', fontsize=11, fontweight='bold')
ax.set_ylabel('Mean Position Error (m)', fontsize=11, fontweight='bold')
ax.set_title('Position Error Distribution by Variant', fontsize=12, fontweight='bold')
ax.grid(True, alpha=0.3, axis='y', linewidth=0.5)
plt.setp(ax.xaxis.get_majorticklabels(), rotation=45, ha='right')

# 2. Heatmap - Mean Position Error by Variant and Test
ax = axes[0, 1]
error_matrix = np.zeros((len(variants), len(tests)))
sorted_tests_list = sorted([str(t) for t in tests])
for i, variant in enumerate(variants):
    for j, test in enumerate(sorted_tests_list):
        mask = (stats_df['variant'] == variant) & (stats_df['test'] == test)
        if mask.any():
            error_matrix[i, j] = stats_df[mask]['mean_pos_error'].values[0]

im = ax.imshow(error_matrix, cmap='RdYlGn_r', aspect='auto')
ax.set_xticks(np.arange(len(sorted_tests_list)))
ax.set_yticks(np.arange(len(variants)))
ax.set_xticklabels(sorted_tests_list)
ax.set_yticklabels(variants)
ax.set_xlabel('Test Number', fontsize=11, fontweight='bold')
ax.set_ylabel('Variant', fontsize=11, fontweight='bold')
ax.set_title('Mean Position Error Heatmap', fontsize=12, fontweight='bold')

cbar = plt.colorbar(im, ax=ax)
cbar.set_label('Mean Error (m)', fontsize=10, fontweight='bold')

# Add text annotations
if error_matrix.size > 0 and len(sorted_tests_list) > 0:
    for i in range(len(variants)):
        for j in range(len(sorted_tests_list)):
            if error_matrix[i, j] > 0:
                ax.text(j, i, f'{error_matrix[i, j]:.3f}',
                       ha="center", va="center", color="black", fontsize=7, fontweight='bold')

# 3. Mean Error by Variant with Error Bars
ax = axes[0, 2]
variant_mean_errors = [stats_df[stats_df['variant'] == v]['mean_pos_error'].mean() for v in variants]
variant_std_errors = [stats_df[stats_df['variant'] == v]['mean_pos_error'].std() for v in variants]
x_pos = np.arange(len(variants))

bars = ax.bar(x_pos, variant_mean_errors, yerr=variant_std_errors, capsize=5,
              alpha=0.75, color=variant_colors, edgecolor='black', linewidth=1.5)
ax.set_xlabel('Variant', fontsize=11, fontweight='bold')
ax.set_ylabel('Mean Position Error (m)', fontsize=11, fontweight='bold')
ax.set_title('Mean Position Error by Variant (±1 SD)', fontsize=12, fontweight='bold')
ax.set_xticks(x_pos)
ax.set_xticklabels(variants)
ax.grid(True, alpha=0.3, axis='y', linewidth=0.5)
plt.setp(ax.xaxis.get_majorticklabels(), rotation=45, ha='right')

# Add value labels
for i, (mean, std) in enumerate(zip(variant_mean_errors, variant_std_errors)):
    ax.text(i, mean + std + 0.002, f'{mean:.3f}m', ha='center', va='bottom',
            fontsize=9, fontweight='bold')

# 4. Box Plot - Orientation Error by Variant
ax = axes[1, 0]
variant_orient_errors = [stats_df[stats_df['variant'] == v]['mean_orient_error'].values for v in variants]
bp = ax.boxplot(variant_orient_errors, tick_labels=variants, patch_artist=True, widths=0.6)

for patch, color in zip(bp['boxes'], variant_colors):
    patch.set_facecolor(color)
    patch.set_alpha(0.7)
    patch.set_edgecolor('black')
    patch.set_linewidth(1.5)

for element in ['whiskers', 'fliers', 'medians', 'caps']:
    plt.setp(bp[element], color='black', linewidth=1.5)

ax.set_xlabel('Variant', fontsize=11, fontweight='bold')
ax.set_ylabel('Mean Orientation Error (rad)', fontsize=11, fontweight='bold')
ax.set_title('Orientation Error Distribution by Variant', fontsize=12, fontweight='bold')
ax.grid(True, alpha=0.3, axis='y', linewidth=0.5)
plt.setp(ax.xaxis.get_majorticklabels(), rotation=45, ha='right')

# 5. Scatter: Duration vs Position Error (colored by variant)
ax = axes[1, 1]
for i, variant in enumerate(variants):
    variant_mask = stats_df['variant'] == variant
    durations = []
    errors = []
    for _, row in stats_df[variant_mask].iterrows():
        # Get duration for this variant-test combination
        dur_mask = (df_duration_summary['variant'] == row['variant']) & (df_duration_summary['test'] == row['test'])
        if dur_mask.any():
            durations.append(df_duration_summary[dur_mask]['duration'].values[0])
            errors.append(row['mean_pos_error'])
    
    if len(durations) > 0:
        ax.scatter(durations, errors, c=[variant_colors[i]], s=100, alpha=0.7,
                  edgecolors='black', linewidth=1.5, label=variant)

ax.set_xlabel('Navigation Duration (s)', fontsize=11, fontweight='bold')
ax.set_ylabel('Mean Position Error (m)', fontsize=11, fontweight='bold')
ax.set_title('Duration vs Position Error by Variant', fontsize=12, fontweight='bold')
ax.legend(fontsize=8, ncol=2, loc='best')
ax.grid(True, alpha=0.3, linewidth=0.5)

# 6. Max Error Comparison by Variant
ax = axes[1, 2]
variant_max_errors = [stats_df[stats_df['variant'] == v]['max_pos_error'].mean() for v in variants]
bars = ax.bar(x_pos, variant_max_errors, alpha=0.75, color=variant_colors,
              edgecolor='black', linewidth=1.5)
ax.set_xlabel('Variant', fontsize=11, fontweight='bold')
ax.set_ylabel('Mean Max Position Error (m)', fontsize=11, fontweight='bold')
ax.set_title('Average Maximum Error by Variant', fontsize=12, fontweight='bold')
ax.set_xticks(x_pos)
ax.set_xticklabels(variants)
ax.grid(True, alpha=0.3, axis='y', linewidth=0.5)
plt.setp(ax.xaxis.get_majorticklabels(), rotation=45, ha='right')

# Add value labels
for i, max_err in enumerate(variant_max_errors):
    ax.text(i, max_err + 0.005, f'{max_err:.3f}m', ha='center', va='bottom',
            fontsize=9, fontweight='bold')

plt.tight_layout()
plt.show()

## Speed Analysis

In [ ]:
# Multi-Variant Speed Analysis
fig, axes = plt.subplots(2, 3, figsize=(20, 12))

# 1. Box Plot - Linear Speed by Variant
ax = axes[0, 0]
variant_linear_speeds = [speed_stats_df[speed_stats_df['variant'] == v]['mean_linear'].values for v in variants]
bp = ax.boxplot(variant_linear_speeds, tick_labels=variants, patch_artist=True, widths=0.6)

for patch, color in zip(bp['boxes'], variant_colors):
    patch.set_facecolor(color)
    patch.set_alpha(0.7)
    patch.set_edgecolor('black')
    patch.set_linewidth(1.5)

for element in ['whiskers', 'fliers', 'medians', 'caps']:
    plt.setp(bp[element], color='black', linewidth=1.5)

ax.set_xlabel('Variant', fontsize=11, fontweight='bold')
ax.set_ylabel('Mean Linear Speed (m/s)', fontsize=11, fontweight='bold')
ax.set_title('Linear Speed Distribution by Variant', fontsize=12, fontweight='bold')
ax.grid(True, alpha=0.3, axis='y', linewidth=0.5)
plt.setp(ax.xaxis.get_majorticklabels(), rotation=45, ha='right')

# 2. Heatmap - Mean Linear Speed by Variant and Test
ax = axes[0, 1]
speed_matrix = np.zeros((len(variants), len(tests)))
sorted_tests_list = sorted([str(t) for t in tests])
for i, variant in enumerate(variants):
    for j, test in enumerate(sorted_tests_list):
        mask = (speed_stats_df['variant'] == variant) & (speed_stats_df['test'] == test)
        if mask.any():
            speed_matrix[i, j] = speed_stats_df[mask]['mean_linear'].values[0]

im = ax.imshow(speed_matrix, cmap='RdYlGn', aspect='auto')
ax.set_xticks(np.arange(len(sorted_tests_list)))
ax.set_yticks(np.arange(len(variants)))
ax.set_xticklabels(sorted_tests_list)
ax.set_yticklabels(variants)
ax.set_xlabel('Test Number', fontsize=11, fontweight='bold')
ax.set_ylabel('Variant', fontsize=11, fontweight='bold')
ax.set_title('Mean Linear Speed Heatmap', fontsize=12, fontweight='bold')

cbar = plt.colorbar(im, ax=ax)
cbar.set_label('Speed (m/s)', fontsize=10, fontweight='bold')

# Add text annotations
for i in range(len(variants)):
    for j in range(len(sorted_tests_list)):
        if speed_matrix[i, j] > 0:
            ax.text(j, i, f'{speed_matrix[i, j]:.2f}',
                   ha="center", va="center", color="black", fontsize=7, fontweight='bold')

# 3. Mean Linear Speed by Variant with Error Bars
ax = axes[0, 2]
variant_mean_speeds = [speed_stats_df[speed_stats_df['variant'] == v]['mean_linear'].mean() for v in variants]
variant_std_speeds = [speed_stats_df[speed_stats_df['variant'] == v]['mean_linear'].std() for v in variants]
x_pos = np.arange(len(variants))

bars = ax.bar(x_pos, variant_mean_speeds, yerr=variant_std_speeds, capsize=5,
              alpha=0.75, color=variant_colors, edgecolor='black', linewidth=1.5)
ax.set_xlabel('Variant', fontsize=11, fontweight='bold')
ax.set_ylabel('Mean Linear Speed (m/s)', fontsize=11, fontweight='bold')
ax.set_title('Mean Linear Speed by Variant (±1 SD)', fontsize=12, fontweight='bold')
ax.set_xticks(x_pos)
ax.set_xticklabels(variants)
ax.grid(True, alpha=0.3, axis='y', linewidth=0.5)
plt.setp(ax.xaxis.get_majorticklabels(), rotation=45, ha='right')

# Add value labels
for i, (mean, std) in enumerate(zip(variant_mean_speeds, variant_std_speeds)):
    if not np.isnan(mean):
        ax.text(i, mean + std + 0.002, f'{mean:.3f}', ha='center', va='bottom',
                fontsize=9, fontweight='bold')

# 4. Box Plot - Angular Speed by Variant
ax = axes[1, 0]
variant_angular_speeds = [speed_stats_df[speed_stats_df['variant'] == v]['mean_angular'].values for v in variants]
bp = ax.boxplot(variant_angular_speeds, tick_labels=variants, patch_artist=True, widths=0.6)

for patch, color in zip(bp['boxes'], variant_colors):
    patch.set_facecolor(color)
    patch.set_alpha(0.7)
    patch.set_edgecolor('black')
    patch.set_linewidth(1.5)

for element in ['whiskers', 'fliers', 'medians', 'caps']:
    plt.setp(bp[element], color='black', linewidth=1.5)

ax.set_xlabel('Variant', fontsize=11, fontweight='bold')
ax.set_ylabel('Mean Angular Speed (rad/s)', fontsize=11, fontweight='bold')
ax.set_title('Angular Speed Distribution by Variant', fontsize=12, fontweight='bold')
ax.grid(True, alpha=0.3, axis='y', linewidth=0.5)
plt.setp(ax.xaxis.get_majorticklabels(), rotation=45, ha='right')

# 5. Scatter: Speed vs Error (colored by variant)
ax = axes[1, 1]
for i, variant in enumerate(variants):
    speeds = []
    errors = []
    for _, row in speed_stats_df[speed_stats_df['variant'] == variant].iterrows():
        # Get error for this variant-test combination
        err_mask = (stats_df['variant'] == row['variant']) & (stats_df['test'] == row['test'])
        if err_mask.any():
            speeds.append(row['mean_linear'])
            errors.append(stats_df[err_mask]['mean_pos_error'].values[0])
    
    if len(speeds) > 0:
        ax.scatter(speeds, errors, c=[variant_colors[i]], s=100, alpha=0.7,
                  edgecolors='black', linewidth=1.5, label=variant)

ax.set_xlabel('Mean Linear Speed (m/s)', fontsize=11, fontweight='bold')
ax.set_ylabel('Mean Position Error (m)', fontsize=11, fontweight='bold')
ax.set_title('Speed vs Position Error by Variant', fontsize=12, fontweight='bold')
ax.legend(fontsize=8, ncol=2, loc='best')
ax.grid(True, alpha=0.3, linewidth=0.5)

# 6. Max Speed Comparison by Variant
ax = axes[1, 2]
variant_max_linear = [speed_stats_df[speed_stats_df['variant'] == v]['max_linear'].mean() for v in variants]
variant_max_angular = [speed_stats_df[speed_stats_df['variant'] == v]['max_angular'].mean() for v in variants]

width = 0.35
x = np.arange(len(variants))
ax.bar(x - width/2, variant_max_linear, width, label='Linear', alpha=0.75,
       color=[c for c in variant_colors], edgecolor='black', linewidth=1.5)
ax.bar(x + width/2, variant_max_angular, width, label='Angular', alpha=0.75,
       color=[c for c in variant_colors], edgecolor='black', linewidth=1.5,
       hatch='//')

ax.set_xlabel('Variant', fontsize=11, fontweight='bold')
ax.set_ylabel('Max Speed', fontsize=11, fontweight='bold')
ax.set_title('Average Maximum Speeds by Variant', fontsize=12, fontweight='bold')
ax.set_xticks(x)
ax.set_xticklabels(variants)
ax.legend(fontsize=10)
ax.grid(True, alpha=0.3, axis='y', linewidth=0.5)
plt.setp(ax.xaxis.get_majorticklabels(), rotation=45, ha='right')

# Add secondary y-axis labels
ax2 = ax.twinx()
ax2.set_ylabel('(Linear: m/s, Angular: rad/s)', fontsize=10, style='italic')
ax2.set_yticks([])

plt.tight_layout()
plt.show()